# Imports

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StringType, StructType, TimestampType, IntegerType

## Cleaning the Data 

Here, I created a function which reads the data from the S3 bucket and loads it into a dataframe.

In [ ]:
def read_s3_mount_data(mount_name):
    ''' A function which reads data from the desired S3 bucket, and returns a 
    usable DataFrame.

    Args:
        mount_name ('string'): Corresponds to the name you gave to the mounted 
        S3 bucket.

    Returns:
        df (`pyspark.sql.DataFrame`): A DataFrame which is ready to be cleaned 
        and queried.
    
    '''

    file_location = f"/mnt/{mount_name}/*.json"
    file_type = "json"
    infer_schema = "true"
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df

Here, we are reading the pin data and cleaning it, I have removed sensitive data too, for example we should have:

```
df_pin = read_s3_mount_data(<mount_name>)
```

In [ ]:
# reading the data
df_pin = read_s3_mount_data(<mount_name>)

# replacing invalid entries with `None`, it is best to define a dictionary 
# here as this makes the whole process more scalable. 

col_and_entries_to_replace = {
    'description' : 'No description available Story format',
    'follower_count' : 'User Info Error',
    'image_src' : 'Image src error.',
    'poster_name' : 'User Info Error',
    'tag_list' : 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e',
    'title' : 'No Title Data Available'
}

for column, value in col_and_entries_to_replace.items():
    df_pin = df_pin.withColumn(column, when(df_pin[column] == value, None).otherwise(df_pin[column]))

# replacing k and M with 000 and 000000 respectivly.
df_pin = (df_pin
    .withColumn('follower_count', 
        when(df_pin.follower_count.endswith('k'), regexp_replace(df_pin.follower_count, 'k', '000'))
        .when(df_pin.follower_count.endswith('M'), regexp_replace(df_pin.follower_count, 'M', '000000'))
        .otherwise(df_pin.follower_count))
)

# casting folower count to integers
df_pin = df_pin.withColumn('follower_count', df_pin.follower_count.cast('int'))

# renaming index to ind so it is consistent with the other tables.
df_pin = df_pin.withColumnRenamed('index', 'ind')

# making the 'save_location' column show the path
df_pin = df_pin.withColumn('save_location', regexp_replace( 'save_location', 'Local save in ', ''))

# reordering columns
df_pin = df_pin.select('ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category')

Here, we are reading the geo data and cleaning it, I have removed sensitive data too, for example we should have:

```
df_geo = read_s3_mount_data(<mount_name>)
```

In [ ]:
# reading the data
df_geo = read_s3_mount_data(<mount_name>)

# creating a new column called 'coordinates'
df_geo = df_geo.withColumn('coordinates', array(df_geo.latitude, df_geo.longitude))

# dropping the columns 'latitude' and 'longitude'
df_geo = df_geo.drop(*['latitude', 'longitude'])

# converting 'timestamp' to a timestamp data type, and formatting it.
df_geo = df_geo.withColumn('timestamp', to_timestamp(df_geo.timestamp))
df_geo = df_geo.withColumn('timestamp', date_format('timestamp', 'yyyy-MM-dd HH:mm:ss'))

# reordering columns
df_geo = df_geo.select('ind','country','coordinates','timestamp')

Here, we are reading the user data and cleaning it, I have removed sensitive data too, for example we should have:

```
df_user = read_s3_mount_data(<mount_name>)
```

In [ ]:
# reading the data
df_user = read_s3_mount_data(<mount_name>)

# creating a new column called 'user_name'
df_user = df_user.withColumn('user_name', concat(df_user.first_name, lit(' '), df_user.last_name))

# dropping the 'first_name' and 'last_name' columns
df_user = df_user.drop(*['first_name', 'last_name'])

# converting 'date_joined' to a timestamp data type, and formatting it.
df_user = df_user.withColumn('date_joined', to_timestamp(df_user.date_joined))
df_user = df_user.withColumn('date_joined', date_format('date_joined', 'yyyy-MM-dd HH:mm:ss'))

# reordering columns
df_user = df_user.select('ind', 'user_name', 'age', 'date_joined')

## Querying the Data
<small>The fun part!</small>

Find the most popular Pinterest category people post to based on their country.

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_geo.createOrReplaceTempView('df_temp_geo')

# the SQL query

#creating a cte
cte_query = """
       WITH ranking_table AS (
         SELECT df_temp_geo.country AS country,
                df_temp_pin.category AS category,
                COUNT(df_temp_pin.category) AS category_count,
                RANK() OVER(PARTITION BY df_temp_geo.country ORDER BY COUNT(df_temp_pin.category) DESC) AS rank
                FROM df_temp_geo
                JOIN df_temp_pin ON df_temp_geo.ind = df_temp_pin.ind
                GROUP BY df_temp_geo.country, df_temp_pin.category
       )
"""
# using the above cte to create the desired output
result_df_1 = spark.sql(cte_query + """
                    SELECT country,
                           category,
                           category_count
                    FROM ranking_table
                    WHERE rank == 1
                      """)

Find how many posts each category had between 2018 and 2022.

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_geo.createOrReplaceTempView('df_temp_geo')

# the SQL query

#creating a cte
cte_query = """
       WITH ranking_table AS (
              SELECT year(df_temp_geo.timestamp) AS post_year,
                     df_temp_pin.category AS category,
                     COUNT(df_temp_pin.category) AS category_count,
                     RANK() OVER (PARTITION BY year(df_temp_geo.timestamp) ORDER BY COUNT(df_temp_pin.category) DESC) AS rank
              FROM df_temp_geo
              JOIN df_temp_pin ON df_temp_geo.ind = df_temp_pin.ind  
              WHERE 2018 <= year(df_temp_geo.timestamp) AND year(df_temp_geo.timestamp) <= 2022
              GROUP BY year(df_temp_geo.timestamp), df_temp_pin.category     
)
"""

result_df_2 = spark.sql(cte_query + """
                     SELECT post_year,
                           category,
                           category_count
                     FROM ranking_table
                     WHERE rank == 1
                      """)

Find the user with most followers with their corresponding country

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_geo.createOrReplaceTempView('df_temp_geo')

# SQL query 
cte_query = """
       WITH ranking_table AS (
              SELECT df_temp_geo.country AS country,
                     df_temp_pin.poster_name AS poster_name,
                     df_temp_pin.follower_count AS follower_count,
                     RANK() OVER(PARTITION BY df_temp_geo.country ORDER BY df_temp_pin.follower_count DESC) AS rank
              FROM df_temp_pin
              JOIN df_temp_geo ON df_temp_geo.ind = df_temp_pin.ind
       )
"""
result_df_3 = spark.sql(cte_query + """
                    SELECT country,
                           poster_name,
                           follower_count
                    FROM ranking_table
                    WHERE rank == 1
                    ORDER BY follower_count DESC
                      """)
                  
result_df_3 = spark.createDataFrame([result_df_3.head(1)[0]])
result_df_3 = result_df_3.drop('poster_name')   

What is the most popular category people post to based on age groups

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_user.createOrReplaceTempView('df_temp_user')

# SQL query 

# creating a cte to clear things up
cte_query = """
    WITH ranking_table AS (
        SELECT 
            category,
            age_group,
            COUNT(category) AS category_count,
            RANK() OVER (PARTITION BY age_group ORDER BY COUNT(category) DESC) AS rank
        FROM (
               SELECT df_temp_pin.category AS category,
                       CASE 
                         WHEN df_temp_user.age BETWEEN 18 AND 24 THEN '18-24'
                         WHEN df_temp_user.age BETWEEN 25 AND 35 THEN '25-35'
                         WHEN df_temp_user.age BETWEEN 36 AND 50 THEN '36-50'
                         ELSE '50+'
                       END AS age_group
               FROM df_temp_user
               JOIN df_temp_pin ON df_temp_pin.ind = df_temp_user.ind
             )
        GROUP BY age_group, category
    )
"""

result_df_4 = spark.sql(cte_query + """
    SELECT age_group,
           category,
           category_count
    FROM ranking_table 
    WHERE rank == 1
""")

What is the median follower count for users in different age groups

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_user.createOrReplaceTempView('df_temp_user')

# SQL query 

result_df_5 = spark.sql("""
             SELECT
                   CASE 
                        WHEN df_temp_user.age BETWEEN 18 AND 24 THEN '18-24'
                        WHEN df_temp_user.age BETWEEN 25 AND 35 THEN '25-35'
                        WHEN df_temp_user.age BETWEEN 36 AND 50 THEN '36-50'
                        ELSE '50+'
                   END AS age_group,
                   PERCENTILE(df_temp_pin.follower_count, 0.5) AS median_follower_count
             FROM df_temp_user
             JOIN df_temp_pin ON df_temp_pin.ind = df_temp_user.ind
             GROUP BY age_group
             ORDER BY age_group ASC
             """)

Find how many users have joined between 2015 and 2020.

In [ ]:
# creating temporary dataframes
df_user.createOrReplaceTempView('df_temp_user')

# SQL query 

result_df_6 = spark.sql("""
                      SELECT year(date_joined) AS year,
                             COUNT(ind) AS number_users_joined
                      FROM df_temp_user
                      WHERE year(date_joined) BETWEEN 2015 AND 2020
                      GROUP BY year(date_joined)
                      """)

Find the median follower count of users have joined between 2015 and 2020.

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_user.createOrReplaceTempView('df_temp_user')

# SQL query 

result_df_7 = spark.sql("""
                      SELECT year(df_temp_user.date_joined) AS year,
                             PERCENTILE(df_temp_pin.follower_count, 0.5) AS median_follower_count
                      FROM df_temp_user
                      JOIN df_temp_pin ON df_temp_pin.ind = df_temp_user.ind
                      WHERE year(date_joined) BETWEEN 2015 AND 2020
                      GROUP BY year(df_temp_user.date_joined)
                      """)

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

In [ ]:
# creating temporary dataframes
df_pin.createOrReplaceTempView('df_temp_pin')
df_user.createOrReplaceTempView('df_temp_user')

# SQL query 

result_df_8 = spark.sql("""
                      SELECT 
                             CASE 
                                WHEN df_temp_user.age BETWEEN 18 AND 24 THEN '18-24'
                                WHEN df_temp_user.age BETWEEN 25 AND 35 THEN '25-35'
                                WHEN df_temp_user.age BETWEEN 36 AND 50 THEN '36-50'
                                ELSE '50+'
                             END AS age_group,
                             year(df_temp_user.date_joined) AS year,
                             PERCENTILE(df_temp_pin.follower_count, 0.5) AS median_follower_count
                      FROM df_temp_user
                      JOIN df_temp_pin ON df_temp_pin.ind = df_temp_user.ind
                      WHERE year(df_temp_user.date_joined) BETWEEN 2015 AND 2020
                      GROUP BY year, age_group
                      ORDER BY age_group ASC, year ASC
                      """)